# **Problem Statement**


Can you differentiate a weed from a crop seedling?

The ability to do so effectively can mean better crop yields and better stewardship of the environment.

The Aarhus University Signal Processing group, in collaboration with University of Southern Denmark, has recently released a dataset containing images of approximately 960 unique plants belonging to 12 species at several growth stages.

We're hosting this dataset as a Kaggle competition in order to give it wider exposure, to give the community an opportunity to experiment with different image recognition techniques, as well to provide a place to cross-pollenate ideas.


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from smallervggnet import SmallerVGGNet
from imutils import paths
import glob
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (100, 100, 3)

data = []
labels = []
Abs_path='content/gdrive/My Drive/Colab Notebooks/CNN/train/'
print("[INFO] loading images...")
for imgPath in list(paths.list_images(Abs_path)):
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imgPath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    label=imgPath.split('\\')[-2]
    labels.append(label)
    

[INFO] loading images...


In [0]:
from sklearn.preprocessing import LabelBinarizer
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.2, random_state=42)

In [0]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
horizontal_flip=True, fill_mode="nearest")

In [ ]:
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

In [0]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
aug.flow(trainX, trainY, batch_size=BS),
validation_data=(testX, testY),
steps_per_epoch=len(trainX) // BS,
epochs=EPOCHS, verbose=1)

[INFO] compiling model...
[INFO] training network...
Epoch 1/100
118/118 [==============================] - 546s 5s/step - loss: 2.3343 - acc: 0.3633 - val_loss: 12.3454 - val_acc: 0.1474
Epoch 2/100
118/118 [==============================] - 543s 5s/step - loss: 1.4678 - acc: 0.5465 - val_loss: 15.1672 - val_acc: 0.0368
Epoch 3/100
118/118 [==============================] - 22123s 187s/step - loss: 1.1118 - acc: 0.6369 - val_loss: 13.9879 - val_acc: 0.1316
Epoch 4/100
118/118 [==============================] - 470s 4s/step - loss: 0.9739 - acc: 0.6922 - val_loss: 13.6957 - val_acc: 0.1474
Epoch 5/100
118/118 [==============================] - 548s 5s/step - loss: 0.7906 - acc: 0.7381 - val_loss: 13.9892 - val_acc: 0.1316
Epoch 6/100
118/118 [==============================] - 618s 5s/step - loss: 0.7378 - acc: 0.7539 - val_loss: 15.3376 - val_acc: 0.0484
Epoch 7/100
118/118 [==============================] - 586s 5s/step - loss: 0.8461 - acc: 0.7319 - val_loss: 15.2711 - val_acc: 0.047

In [0]:
# save the model to disk
print("[INFO] serializing network...")
model.save('Plant_classifier.model')
 
# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open('labels.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

In [0]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(args["plot"])